In [ ]:
import getpass
import os

if not os.getenv("NVIDIA_API_KEY"):
    # Note: the API key should start with "nvapi-"
    os.environ["NVIDIA_API_KEY"] = getpass.getpass("Enter your NVIDIA API key: ")   ## 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

In [ ]:
%pip install --upgrade --quiet langchain-nvidia-ai-endpoints

In [ ]:
## Core LC Chat Interface
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")
llm

ChatNVIDIA(base_url='https://integrate.api.nvidia.com/v1', model='mistralai/mixtral-8x7b-instruct-v0.1')

In [ ]:
result = llm.invoke("Write a ballad about LangChain.")
print(result.content)

(Verse 1)
In a world of code and data, a hero did rise,
A chain named Lang, with knowledge as its prize.
A quest to translate, to bridge the gap,
Bringing understanding to every lap.

Through the day and night, it worked and learned,
A new dawn of communication, to the world it yearned.
From text and speech, to images and more,
LangChain's might made learning a chore no more.

(Chorus)
Oh, LangChain, strong and wise,
A beacon of hope, in data's guise.
Your power to unite, to break down walls,
Is a ballad we sing, for one and all.

(Verse 2)
A chain of language, in a world so vast,
Connecting minds, through time and space at last.
With every symbol, and every byte,
LangChain illuminates the darkest night.

From simple phrases, to deepest thoughts,
Through the power of its linguistic craft,
A symphony of voices, in harmony join,
To celebrate LangChain's victory, we sing this ballad in lore.

(Bridge)
A song of learning, a hymn of growth,
For LangChain, defender of knowledge's troth.
Brea

In [ ]:
%pip install -q crewai_tools

In [ ]:
%pip install -q crewai

In [ ]:
import os

# Set a dummy OpenAI API key to bypass validation
os.environ["OPENAI_API_KEY"] = "xyz"


In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from crewai import Agent, Task

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [ ]:
# Define the Venue Coordinator agent
rental_coordinator = Agent(
   role="Hotel/Rental Coordinator",
   goal="Identify an appropriate hotel, rental home, or vacation rental.",
   tools=[search_tool, scrape_tool],
   verbose=True,
   backstory=(
     "With a keen sense of space and understanding of vacation logistics, "
     "you excel at finding and securing the perfect vacation rental that fits "
     "the vacation's location, start and end dates, group size, and budget constraints."
    )
)

In [ ]:
# Define the Itinerary Planner agent
itinerary_planner = Agent(
   role="Itinerary Planner",
   goal="Create a proposed itinerary including daily excursions and activities.",
   tools=[search_tool, scrape_tool],
   verbose=True,
   backstory=(
     "With a passion for adventure and local culture, you specialize in planning engaging "
     "and budget-friendly itineraries, taking into account the traveler's interests, budget, "
     "and logistics like transportation needs."
    )
)

In [ ]:
# Define the Budgeting Agent
budgeting_agent = Agent(
   role="Budgeting Agent",
   goal="Manage the overall budget for the trip, considering the cost of accommodation and daily activities.",
   tools=[search_tool],
   verbose=True,
   backstory=(
     "With a knack for financial planning, you ensure the vacation remains within budget while maximizing value and enjoyment."
   )
)

### Tasks

In [ ]:
from crewai import Task
from pydantic import BaseModel
from datetime import date
from typing import List


In [ ]:


# Define a Pydantic model for venue details
class RentalDetails(BaseModel):
   name: str
   address: str
   capacity: int
   price_per_night: int
   start_date: date
   end_date: date
   description: str
   amenities: list[str]
   source: str


# Example task using the tools
hotel_task = Task(
   description="Find a hotel or rental in {vacation_city} that meets criteria for {vacation_details}, {budget}, {group_size}, {start_date}, and {end_date}.",
   expected_output="Details of suitable rental options, including name, address, capacity, price per night, available dates, description, and amenities.",
   human_input=True,
   output_json=RentalDetails,
   output_file="venue_details.json",
   agent=rental_coordinator
)


In [ ]:
# Define a Pydantic model for itinerary details
class ItineraryDetail(BaseModel):
    date: date
    activity: str
    location: str
    description: str
    cost_estimate: int  # Optional: estimate of cost for the activity

# Define a wrapper model that contains a list of itinerary details
class ItineraryDetailsWrapper(BaseModel):
    details: List[ItineraryDetail]

# Define the itinerary planning task
itinerary_task = Task(
    description=(
        "Create a detailed itinerary for {vacation_city} from {start_date} to {end_date}. "
        "Include daily excursions and activities that align with {vacation_details}, "
        "and consider group size {group_size} and budget {budget}. Ensure to highlight "
        "activities that reflect local culture and provide a mix of relaxation and adventure."
    ),
    expected_output="Details of the daily itinerary, including date, activity, location, description, and cost estimate.",
    human_input=True,
    output_json=ItineraryDetailsWrapper,
    output_file="itinerary_details.json",
    agent=itinerary_planner
)


In [ ]:
# Define a Pydantic model for budget details
class BudgetDetails(BaseModel):
    total_budget: int
    accommodation_cost: int
    activities_cost: int
    remaining_budget: int
    notes: str

# Define the budgeting task
budgeting_task = Task(
    description=(
        "Analyze the overall budget for the trip. Calculate the total estimated cost based on "
        "accommodation, daily activities, and any additional expenses. Ensure the total does not "
        "exceed the given budget of {budget} and provide recommendations if necessary."
    ),
    expected_output="Detailed budget breakdown, including total budget, accommodation cost, activities cost, remaining budget, and notes.",
    human_input=True,
    output_json=BudgetDetails,
    output_file="budget_details.json",
    agent=budgeting_agent
)

###  manage the Crew

In [ ]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Example data for kicking off the process
vacation_details = {
   'vacation_city': "Mumbai",
   'vacation_details': "A vacation for an adventurous family of 7 who want to explore Mumbai, see the nature, and experience some good food and culture",
   'start_date': "2024-09-15",
   'end_date': "2024-09-22",
   'group_size': 7,
   'budget': 100000,
}

# Define the crew with the venue coordinator agent, itinerary planner agent, and budgeting agent
vacation_planning_crew = Crew(
   agents=[rental_coordinator, itinerary_planner, budgeting_agent],
   tasks=[hotel_task, itinerary_task, budgeting_task],
   manager_llm=llm,
   process=Process.sequential,
   verbose=True
)

In [ ]:
result = vacation_planning_crew.kickoff(inputs=vacation_details)
result

 [2024-09-04 05:52:18][DEBUG]: == Working Agent: Hotel/Rental Coordinator
 [2024-09-04 05:52:18][INFO]: == Starting Task: Find a hotel or rental in Mumbai that meets criteria for A vacation for an adventurous family of 7 who want to explore Mumbai, see the nature, and experience some good food and culture, 100000, 7, 2024-09-15, and 2024-09-22.


> Entering new CrewAgentExecutor chain...


> Finished chain.
 Error parsing JSON: Expecting value: line 1 column 1 (char 0). Attempting to handle partial JSON.
 Failed to convert text into JSON, error: Error code: 401 - {'error': {'message': 'Incorrect API key provided: xyz. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Using raw output instead.
 [2024-09-04 05:52:26][DEBUG]: == [Hotel/Rental Coordinator] Task output: Agent stopped due to iteration limit or time limit.


 [2024-09-04 05:52:26][DEBUG]: == Working Agent: Itinerary Planner

CrewOutput(raw='Agent stopped due to iteration limit or time limit.', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Find a hotel or rental in Mumbai that meets criteria for A vacation for an adventurous family of 7 who want to explore Mumbai, see the nature, and experience some good food and culture, 100000, 7, 2024-09-15, and 2024-09-22.', summary='Find a hotel or rental in Mumbai that meets criteria...', raw='Agent stopped due to iteration limit or time limit.', pydantic=None, json_dict=None, agent='Hotel/Rental Coordinator', output_format=<OutputFormat.JSON: 'json'>), TaskOutput(description='Create a detailed itinerary for Mumbai from 2024-09-15 to 2024-09-22. Include daily excursions and activities that align with A vacation for an adventurous family of 7 who want to explore Mumbai, see the nature, and experience some good food and culture, and consider group size 7 and budget 100000. Ensure to highlight activities that reflect local culture and provide a mix

In [ ]:
%pip install python-docx

In [ ]:
import os
from crewai import Agent, Task, Crew, Process
from docx import Document

# Define the agents
introduction_agent = Agent(
    role='Introduction Writer',
    goal='Gather information for the introduction section of the SRS',
    backstory='You are responsible for collecting and writing the introduction of the SRS document.',
)

overall_description_agent = Agent(
    role='Overall Description Writer',
    goal='Gather information for the overall description section of the SRS',
    backstory='You focus on describing the product and its environment in the SRS document.',
)

specific_requirements_agent = Agent(
    role='Requirements Gatherer',
    goal='Gather specific functional and non-functional requirements for the SRS',
    backstory='You are responsible for detailing the functional and non-functional requirements.',
)

# Define the tasks
introduction_task = Task(
    description=(
        "Collect the following details:\n"
        "1. Purpose\n"
        "2. Scope\n"
        "3. Definitions, Acronyms, and Abbreviations\n"
        "4. References\n"
        "5. Overview"
    ),
    expected_output='Introduction section details collected.',
    agent=introduction_agent,
)

overall_description_task = Task(
    description=(
        "Collect the following details for the Overall Description:\n"
        "1. Product Perspective\n"
        "2. Product Functions\n"
        "3. User Classes and Characteristics\n"
        "4. Operating Environment\n"
        "5. Design and Implementation Constraints\n"
        "6. Assumptions and Dependencies"
    ),
    expected_output='Overall Description section details collected.',
    agent=overall_description_agent,
)

specific_requirements_task = Task(
    description=(
        "Collect the following details for the Specific Requirements:\n"
        "1. Functional Requirements\n"
        "2. Performance Requirements\n"
        "3. Logical Database Requirements\n"
        "4. Design Constraints\n"
        "5. Software System Attributes"
    ),
    expected_output='Specific Requirements section details collected.',
    agent=specific_requirements_agent,
)

# Combine the agents and tasks into a crew
srs_crew = Crew(
    agents=[introduction_agent, overall_description_agent, specific_requirements_agent],
    tasks=[introduction_task, overall_description_task, specific_requirements_task],
    process=Process.sequential,
    manager_llm=llm
)

# Kickoff the process and gather the results
def kickoff_srs_crew():
    # Start the crew process
    results = srs_crew.kickoff()

    # # Create the SRS document
    # doc = Document()
    # doc.add_heading('Software Requirements Specification (SRS)', 0)

    # # Add sections based on agent results
    # doc.add_heading('1. Introduction', level=1)
    # doc.add_paragraph(results[introduction_task])

    # doc.add_heading('2. Overall Description', level=1)
    # doc.add_paragraph(results[overall_description_task])

    # doc.add_heading('3. Specific Requirements', level=1)
    # doc.add_paragraph(results[specific_requirements_task])

    # # Save the document
    # file_name = 'SRS_Document'
    # doc.save(f'{file_name}.docx')
    # print(f"SRS document '{file_name}.docx' created successfully.")
    print(results)

# Execute the crew
kickoff_srs_crew()


Agent stopped due to iteration limit or time limit.
